<a href="https://colab.research.google.com/github/sisomimoctrl/study-pytorch/blob/main/simple_linear_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from enum import Enum

torch.manual_seed(777)

"""
 0: 기본
 1: nn.Linear 사용
 2: model을 클래스로 구현
"""
method = 2

print(f'method : {method}\n')

# 데이터
x_train = torch.FloatTensor([2,4,6]).unsqueeze(1) #옆과 같음. x_train = torch.FloatTensor([[2],[4],[6]])
y_train = torch.FloatTensor([8,16,24]).unsqueeze(1) #옆과 같음. y_train = torch.FloatTensor([[8],[16],[24]]) 

if method == 0:
    # 모델을 선언 및 초기화. y_train = x_train*W + b
    W = torch.zeros(1, requires_grad=True)
    b = torch.zeros(1, requires_grad=True)
    
    # optimizer 설정. 경사 하강법 SGD를 사용하고 learning rate를 의미하는 lr은 0.01
    optimizer = optim.SGD([W, b], lr=0.01)

elif method == 1:
    # 모델을 선언 및 초기화. 단순 선형 회귀이므로 input_dim=1, output_dim=1.
    model = nn.Linear(1,1)

    # optimizer 설정. 경사 하강법 SGD를 사용하고 learning rate를 의미하는 lr은 0.01
    optimizer = torch.optim.SGD(model.parameters(), lr=0.01) 
    
elif method == 2:
    class LinearRegressionModel(nn.Module): # torch.nn.Module을 상속받는 파이썬 클래스
        def __init__(self):
            super().__init__()
            self.linear = nn.Linear(1, 1) # 단순 선형 회귀이므로 input_dim=1, output_dim=1

        def forward(self, x):
            return self.linear(x)

    # 모델을 선언 및 초기화. 단순 선형 회귀이므로 input_dim=1, output_dim=1.
    model = LinearRegressionModel()

    # optimizer 설정. 경사 하강법 SGD를 사용하고 learning rate를 의미하는 lr은 0.01
    optimizer = torch.optim.SGD(model.parameters(), lr=0.01) 
        
        
# 전체 훈련 데이터에 대해 경사 하강법을 3,000회 반복
nb_epochs = 3000
for epoch in range(nb_epochs + 1):

    if method == 0:
        # H(x) 계산
        hypothesis = x_train * W + b
        
        # cost 계산
        cost = torch.mean((hypothesis - y_train)**2)
        
    else: # method=1, 2
        # H(x) 계산
        prediction = model(x_train)
        
        # cost 계산
        cost = F.mse_loss(prediction, y_train)

        
    # cost로 H(x) 개선하는 부분
    # gradient를 0으로 초기화
    optimizer.zero_grad()
    # 비용 함수를 미분하여 gradient 계산
    cost.backward()
    # W와 b를 업데이트
    optimizer.step()
    
    if epoch % 300 == 0:
        print(f'Epoch {epoch:4d}/{nb_epochs} Cost: {cost.item():.6f}')
        
# 검증과정

# 임의로 입력 8를 선언
new_var =  torch.FloatTensor([[8.0]]) 

# 입력한 값 8에 대해서 예측값 y를 리턴받아서 pred_y에 저장
# 파라미터 출력
if method == 0:
    pred_y = new_var*W+b # forward 연산
    print(f'\nW: {W}, b: {b}\n')
else:
    pred_y = model(new_var) # forward 연산
    print(f'\n{list(model.parameters())}\n')

# y = 4x 이므로 입력이 8라면 y가 32에 가까운 값이 나와야 제대로 학습이 된 것
print(f'훈련 후 입력이 {new_var.item()}일 때의 예측값 :{pred_y.item():.6f}')

method : 2

Epoch    0/3000 Cost: 437.272064
Epoch  300/3000 Cost: 0.026633
Epoch  600/3000 Cost: 0.005162
Epoch  900/3000 Cost: 0.001001
Epoch 1200/3000 Cost: 0.000194
Epoch 1500/3000 Cost: 0.000038
Epoch 1800/3000 Cost: 0.000007
Epoch 2100/3000 Cost: 0.000001
Epoch 2400/3000 Cost: 0.000000
Epoch 2700/3000 Cost: 0.000000
Epoch 3000/3000 Cost: 0.000000

[Parameter containing:
tensor([[3.9999]], requires_grad=True), Parameter containing:
tensor([0.0003], requires_grad=True)]

훈련 후 입력이 8.0일 때의 예측값 :31.999805
